## 필요 라이브러리 로드

In [35]:
import ifcopenshell
import ifcopenshell.geom
import ifcopenshell.util.shape
import ifcopenshell.util.selector
import numpy as np
import multiprocessing
import math
import open3d as o3d


## ifc 파일 로드

In [36]:
ifc_file = ifcopenshell.open('sample.ifc')
print('ifc_file : ', ifc_file)

ifc_file :  <ifcopenshell.file.file object at 0x0000012EFAE4B6D0>


## Target 객체 찾기

In [37]:
list_of_target = ifcopenshell.util.selector.filter_elements(ifc_file, "My_Data.cnv_class=target")
print(list_of_target)

{#1220=IfcWall('3Grk8JCXL0Wv356Wrvw3Rc',$,'Wall',$,$,#1866,#1228,$,$)}


## Target 객체 위치 좌표점 가져오기

In [ ]:
# 변수 설정
RAY_COUNT = 360
RAY_LENGTH = 50 #(m기준)


--------- 1 번째 Target객체 ---------
Name :  Wall
Location: (0.0, 0.0, 0.5)
(1.0, 0.0, 0.1)
(0.985, 0.174, 0.1)
(0.94, 0.342, 0.1)
(0.866, 0.5, 0.1)
(0.766, 0.643, 0.1)
(0.643, 0.766, 0.1)
(0.5, 0.866, 0.1)
(0.342, 0.94, 0.1)
(0.174, 0.985, 0.1)
(0.0, 1.0, 0.1)
(-0.174, 0.985, 0.1)
(-0.342, 0.94, 0.1)
(-0.5, 0.866, 0.1)
(-0.643, 0.766, 0.1)
(-0.766, 0.643, 0.1)
(-0.866, 0.5, 0.1)
(-0.94, 0.342, 0.1)
(-0.985, 0.174, 0.1)
(-1.0, 0.0, 0.1)
(-0.985, -0.174, 0.1)
(-0.94, -0.342, 0.1)
(-0.866, -0.5, 0.1)
(-0.766, -0.643, 0.1)
(-0.643, -0.766, 0.1)
(-0.5, -0.866, 0.1)
(-0.342, -0.94, 0.1)
(-0.174, -0.985, 0.1)
(-0.0, -1.0, 0.1)
(0.174, -0.985, 0.1)
(0.342, -0.94, 0.1)
(0.5, -0.866, 0.1)
(0.643, -0.766, 0.1)
(0.766, -0.643, 0.1)
(0.866, -0.5, 0.1)
(0.94, -0.342, 0.1)
(0.985, -0.174, 0.1)
간섭 수: 13


In [ ]:

# geom 세팅 생성
settings = ifcopenshell.geom.settings()
settings.set(settings.USE_WORLD_COORDS, True)

# geom tree (ray) 세팅
tree = ifcopenshell.geom.tree()
settings = ifcopenshell.geom.settings()
iterator = ifcopenshell.geom.iterator(settings, ifc_file, multiprocessing.cpu_count())
if iterator.initialize():
    while True:
        # Use triangulation to build a BVH tree
        # tree.add_element(iterator.get())

        # Alternatively, use this code to build an unbalanced binary tree
        tree.add_element(iterator.get_native())

        if not iterator.next():
            break

# Target 객체 반복
i = 1
cross_list_list =[]
for element in list_of_target:
    print("---------",i,"번째 Target객체 ---------")
    print("Name : ", element.Name)
    # shape 가져오기
    shape = ifcopenshell.geom.create_shape(settings, element)
    geometry = shape.geometry
    matrix = ifcopenshell.util.shape.get_shape_matrix(shape)
    location = matrix[:,3][0:3]    
    verts = np.array(geometry.verts).reshape(-1,3)
    
    # 튜플 형태로 변환 (예: (0., 0., 0.))
    location_tuple = tuple(map(float, location))
    print("Location:", location_tuple)

    # ray 반복
    cross_count = 0

    cross_list = []
    for j in range(RAY_COUNT):
        angle_deg = j * (360 / RAY_COUNT)
        angle_rad = math.radians(angle_deg)

        # direction 벡터 계산 (XY 평면에서 Z는 0)
        direction = (math.cos(angle_rad), math.sin(angle_rad), 0.1)
        print(tuple(round(x, 3) for x in direction))
        # 정규화 (단위 벡터 보장)
        norm = math.sqrt(direction[0]**2 + direction[1]**2)
        direction = (direction[0]/norm, direction[1]/norm, 0.1)

        # ray 쏘기
        results = tree.select_ray(location_tuple, direction, length=RAY_LENGTH)
        if len(results) > 0:
            cross_list.append(1)
            cross_count += 1
        else:
            cross_list.append(0)
    cross_list_list.append(cross_list)
    print('간섭 수:', cross_count)

    # for result in results:
    #     print(ifc_file.by_id(result.instance.id())) # The element the ray intersects with
    #     # print(list(result.position)) # The XYZ intersection point
    #     print(result.distance) # The distance between the ray origin and the intersection
    #     # print(list(result.normal)) # The normal of the face being intersected
    #     print(result.dot_product) # The dot product of the face being intersected with the ray



    

## 시각화 테스트

In [39]:
import open3d as o3d

# 👉 ray 시각화용 데이터 초기화
ray_lines = []
ray_points = []
line_colors = []

# 기준점 (numpy 배열로 변환)
origin_np = np.array(location_tuple)

for j in range(RAY_COUNT):
    angle_deg = j * (360 / RAY_COUNT)
    angle_rad = math.radians(angle_deg)

    # 방향 벡터
    direction = np.array([math.cos(angle_rad), math.sin(angle_rad), 0.1])
    direction /= np.linalg.norm(direction)

    # ray 끝점
    end = origin_np + direction * RAY_LENGTH

    # 점 추가 (시작점, 끝점)
    ray_points.append(origin_np)
    ray_points.append(end)

    # 선 인덱스
    ray_lines.append([2 * j, 2 * j + 1])

    # 선 색상 (빨간색)
    if cross_list_list[0][j] == 0:
        line_colors.append([0,0,0])
    else:
        line_colors.append([1, 0, 0])

# 시각화용 라인셋 생성
line_set = o3d.geometry.LineSet(
    points=o3d.utility.Vector3dVector(ray_points),
    lines=o3d.utility.Vector2iVector(ray_lines),
)
line_set.colors = o3d.utility.Vector3dVector(line_colors)

# ✅ 시각화 실행 (중심점 제거됨)
o3d.visualization.draw_geometries([line_set])


## 전체 ray객체 기준 distraction 간섭 객체개수 % 결과값 확인

# TEST
